бенчмарк методов декомпозиции

# Imports

In [64]:
import sys

from src.utils.eeg_controller import get_eegbci_dataset

modules_to_reload = [
    "src.utils.method_loggers",
    "src.utils.method_runners",
    "src.utils.metrics_calculators",
    "src.utils.tensor_handlers",
    "src.utils.trackers",
    "src.utils.video_controller",
    "src.utils.optimal_rank_finders",
    "src.utils.benchmark.calculate_optimized_rank_tucker",
    "src.utils.benchmark.calculate_optimized_rank_tensor_train",
]

for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

import contextlib
import gc
import os
from itertools import product

import numpy as np
import tensorflow as tf
from numba import cuda

np.random.seed(42)
os.environ["OPENBLAS_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"

tf.random.set_seed(42)

import t3f
import tensorly as tl
import torch
from dotenv import load_dotenv
from tqdm import tqdm

from src.utils.eeg_controller import create_eeg_limo_data_tensor
from src.utils.image_controller import download_image, extract_image_frames
from src.utils.method_loggers import MethodLogger
from src.utils.method_runners import MethodRunner
from src.utils.optimal_rank_finders import (
    find_optimal_rank_tensor_train_by_compression_ratio,
    find_optimal_rank_tucker_by_compression_ratio,
)
from src.utils.read_logs import LogReader
from src.utils.save_frames import SaveFramesFactory
from src.utils.tensor_handlers import normalize_frames
from src.utils.trackers import (
    GPUTensorflowMemoryTracker,
    GPUTorchMemoryTracker,
    RAMMemoryTracker,
    TimeTracker,
)
from src.utils.video_controller import download_youtube_video, extract_frames
from src.utils.benchmark.calculate_optimized_rank_tucker import global_optimize_tucker_rank
from src.utils.benchmark.calculate_optimized_rank_tensor_train import global_optimize_tensor_train_rank

load_dotenv()

torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


3106

In [65]:
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_device

device(type='cuda')

In [66]:
torch.cuda.empty_cache()

In [67]:
tf_physical_device = tf.config.list_physical_devices("GPU")[0].name
tf_device = ":".join(tf_physical_device.split(":")[1:3])
tf_devices = [tf_device]
tf_physical_device

'/physical_device:GPU:0'

# Get tensors

## Some params

In [68]:
proxy_url = os.getenv("PROXY_URL")
log_file_path = "../.cache/method_logs.json"
method_logs_list = []

## Get some tensors from different data types

### Video

In [69]:
cache_dir_video = "../.cache/video"

video_urls = [
    "https://www.youtube.com/watch?v=eSKe2Vx-rpY",
    "https://www.youtube.com/watch?v=zk1mAd77Hr4",
    "https://www.youtube.com/watch?v=vSLHsTh421w",
]

videos = {}

In [70]:
video_paths = [
    download_youtube_video(
        video_url=video_url,
        cache_dir=cache_dir_video.__str__(),
        proxy_url=proxy_url,
    )
    for video_url in video_urls
]

Видео уже загружено и закешировано: ../.cache/video/eSKe2Vx-rpY.mp4
Видео уже загружено и закешировано: ../.cache/video/zk1mAd77Hr4.mp4
Видео уже загружено и закешировано: ../.cache/video/vSLHsTh421w.mp4


In [71]:
for video_index, video_path in enumerate(video_paths):
    video_frames, original_fps, frame_size = extract_frames(video_path)

    videos[f"video-{video_index}"] = {
        "video_url": video_urls[video_index],
        "video_path": video_path,
        "original_fps": original_fps,
        "frame_size": frame_size,
        "frames": video_frames,
    }

    print(f"video-{video_index} - {video_frames.shape}")

video-0 - (220, 256, 144, 3)
video-1 - (100, 144, 192, 3)
video-2 - (237, 144, 256, 3)


### Image

In [72]:
cache_dir_image = "../.cache/image"

image_urls = [
    "https://i.pinimg.com/564x/04/b2/68/04b26838bdd5e2ba54d0144558685bae.jpg",
    "https://cdnstatic.rg.ru/crop620x412/uploads/images/187/94/47/iStock-644032024.jpg",
    "https://i.sstatic.net/uQggz.png",
]

images = {}

In [73]:
image_paths = [download_image(image_url, cache_dir_image) for image_url in image_urls]

Изображение уже загружено и закешировано: ../.cache/image/04b26838bdd5e2ba54d0144558685bae.jpg
Изображение уже загружено и закешировано: ../.cache/image/iStock-644032024.jpg
Изображение уже загружено и закешировано: ../.cache/image/uQggz.png


In [74]:
for image_index, image_path in enumerate(image_paths):
    image_frames = extract_image_frames(image_path)

    images[f"image-{image_index}"] = {
        "image_url": image_urls[image_index],
        "image_path": image_path,
        "frames": image_frames,
    }

    print(f"image-{image_index} - {image_frames.shape}")

image-0 - (564, 564, 3)
image-1 - (412, 620, 3)
image-2 - (689, 1195, 3)


In [75]:
# from src.utils.save_frames import SaveFramesAsImage
#
# SaveFramesAsImage.save_frames('test', images['image-2']['frames'])

### EEG

In [76]:
cache_dir_eeg = "../.cache/eeg"

eegs = {
    "eeg-0": {"frames": get_eegbci_dataset(cache_dir_eeg=cache_dir_eeg)},
    "eeg-1": {"frames": create_eeg_limo_data_tensor(cache_dir_eeg=cache_dir_eeg)},
}

Extracting EDF parameters from /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/eeg/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/eeg/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/eeg/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/eeg/MNE-eegbci-data/file

In [77]:
for index, eeg in eegs.items():
    print(f"{index} - {eeg['frames'].shape}")

eeg-0 - (4, 12, 2, 15, 64, 1281)
eeg-1 - (3, 1050, 2, 132, 201)


## Find optimal ranks

### Params

In [78]:
target_compression_ratio = 50.0
frobenius_error_coef = 1.0
compression_ratio_coef = 10.0

### Tucker

In [79]:
tucker_args = {
    "svd": "truncated_svd",
    "init": "svd",  # random, svd
    "random_state": 42,
}

In [80]:
input_tucker_tensors = {
    "image-0": images["image-0"]["frames"].copy().astype(np.float32),
    "image-1": images["image-1"]["frames"].copy().astype(np.float32),
    "image-2": images["image-2"]["frames"].copy().astype(np.float32),
    "video-0": videos["video-0"]["frames"].copy().astype(np.float32),
    "video-1": videos["video-1"]["frames"].copy().astype(np.float32),
    "video-2": videos["video-2"]["frames"].copy().astype(np.float32),
    "eeg-0": eegs["eeg-0"]["frames"].copy().astype(np.float32),
    "eeg-1": eegs["eeg-1"]["frames"].copy().astype(np.float32),
}

tensor_tucker_rank_mapping = {
    "image-0": [252, 255, 3],
    "image-1": [220, 231, 3],
    "image-2": [212, 595, 3],
    "video-0": [176, 205, 112, 3],
    "video-1": [51, 144, 186, 3],
    "video-2": [222, 110, 178, 3],
    "eeg-0": [4, 12, 2, 8, 63, 1221],
    "eeg-1": [3, 1050, 2, 131, 100],
    
    # old with custom local optimization algorithm
    # "image-0": [252, 254, 3],
    # "image-1": [216, 232, 3],
    # "image-2": [261, 534, 3],
    # "video-0": [173, 194, 120, 3],
    # "video-1": [58, 143, 165, 3],
    # "video-2": [219, 115, 172, 3],
    # "eeg-0": [4, 12, 2, 8, 64, 1171],
    # "eeg-1": [3, 1046, 1, 132, 199],
}

In [81]:
for tensor_name, tensor_data in input_tucker_tensors.items():
    with tl.backend_context("pytorch"):
        try:
            if tensor_name not in tensor_tucker_rank_mapping:
                print(f"Processing tensor: {tensor_name}")
                method = "differential_evolution"

                try:
                    reconstructed_tensor, weight, factors, optimal_rank, final_loss_value, optimize_result, iteration_logs = (
                        global_optimize_tucker_rank(
                            optimization_method=method,
                            tensor=tensor_data,
                            target_compression_ratio=target_compression_ratio,
                            frobenius_error_coef=frobenius_error_coef,
                            compression_ratio_coef=compression_ratio_coef,
                            verbose=True,
                        )
                    )

                    tensor_tucker_rank_mapping[tensor_name] = optimal_rank

                    frobenius_error = iteration_logs[-1]['metrics']['frobenius_error'] * 100
                    compression_ratio = iteration_logs[-1]['metrics']['compression_ratio'] * 100

                    print(
                        f"Tensor {tensor_name}: Best Rank = {optimal_rank}. Last iter results: Frobenius Error = {frobenius_error:.4f}%, Compression Ratio = {compression_ratio:.4f}%"
                    )
                except Exception as e:
                    print(f"Error with method {method}: {e}")
            else:
                print(f"Skipping tensor: {tensor_name} cause rank is exist")
        except Exception as e:
            print(f"Error processing tensor {tensor_name}: {e}")
        finally:
            torch.cuda.empty_cache()
            gc.collect()

Skipping tensor: image-0 cause rank is exist
Skipping tensor: image-1 cause rank is exist
Skipping tensor: image-2 cause rank is exist
Skipping tensor: video-0 cause rank is exist
Skipping tensor: video-1 cause rank is exist
Skipping tensor: video-2 cause rank is exist
Skipping tensor: eeg-0 cause rank is exist
Skipping tensor: eeg-1 cause rank is exist


In [82]:
for tensor_name, tensor in input_tucker_tensors.items():
    print(tensor_name, tensor.shape)

image-0 (564, 564, 3)
image-1 (412, 620, 3)
image-2 (689, 1195, 3)
video-0 (220, 256, 144, 3)
video-1 (100, 144, 192, 3)
video-2 (237, 144, 256, 3)
eeg-0 (4, 12, 2, 15, 64, 1281)
eeg-1 (3, 1050, 2, 132, 201)


In [83]:
for tensor_name, rank in tensor_tucker_rank_mapping.items():
    print(tensor_name, rank)

image-0 [252, 255, 3]
image-1 [220, 231, 3]
image-2 [212, 595, 3]
video-0 [176, 205, 112, 3]
video-1 [51, 144, 186, 3]
video-2 [222, 110, 178, 3]
eeg-0 [4, 12, 2, 8, 63, 1221]
eeg-1 [3, 1050, 2, 131, 100]


### Tensor Train

In [84]:
tensor_train_args = {"svd": "truncated_svd"}

In [85]:
input_tensor_train_tensors = {
    "image-0": images["image-0"]["frames"].copy().astype(np.float32),
    "image-1": images["image-1"]["frames"].copy().astype(np.float32),
    "image-2": images["image-2"]["frames"].copy().astype(np.float32),
    "video-0": videos["video-0"]["frames"].copy().astype(np.float32),
    "video-1": videos["video-1"]["frames"].copy().astype(np.float32),
    "video-2": videos["video-2"]["frames"].copy().astype(np.float32),
    "eeg-0": eegs["eeg-0"]["frames"].copy().astype(np.float32),
    "eeg-1": eegs["eeg-1"]["frames"].copy().astype(np.float32),
}

tensor_tensor_train_rank_mapping = {
    "image-0": [1, 212, 3, 1],
    "image-1": [1, 233, 2, 1],
    "image-2": [1, 289, 3, 1],
    "video-0": [1, 213, 222, 3, 1],
    "video-1": [1, 83, 331, 3, 1],
    "video-2": [1, 237, 374, 3, 1],
    "eeg-0": [1, 4, 48, 96, 720, 1280, 1],
    "eeg-1": [1, 3, 3150, 3152, 129, 1],
    
    # old
    # "image-0": [1, 212, 3, 1],
    # "image-1": [1, 169, 3, 1],
    # "image-2": [1, 289, 3, 1],
    # "video-0": [1, 220, 215, 1, 1],
    # "video-1": [1, 100, 277, 3, 1],
    # "video-2": [1, 237, 374, 3, 1],
    # "eeg-0": [1, 4, 48, 2, 30, 1282, 1],
}

# eeg-1 - (3, 1050, 2, 132, 201)
# 11h eeg-1 1795 | [1, 3, 1532, 217, 47, 1] | 4.095930 % | 44.831035 %

In [86]:
for tensor_name, tensor_data in input_tensor_train_tensors.items():
    with tl.backend_context("pytorch"):
        try:
            if tensor_name not in tensor_tensor_train_rank_mapping:
                print(f"Processing tensor: {tensor_name}")
                method = "differential_evolution"

                try:
                    reconstructed_tensor, tt_factors, optimal_rank, final_loss_value, optimize_result, iteration_logs = (
                        global_optimize_tensor_train_rank(
                            optimization_method=method,
                            tensor=tensor_data,
                            target_compression_ratio=target_compression_ratio,
                            frobenius_error_coef=frobenius_error_coef,
                            compression_ratio_coef=compression_ratio_coef,
                            verbose=True,
                        )
                    )

                    tensor_tensor_train_rank_mapping[tensor_name] = optimal_rank

                    frobenius_error = iteration_logs[-1]['metrics']['frobenius_error'] * 100
                    compression_ratio = iteration_logs[-1]['metrics']['compression_ratio'] * 100

                    print(
                        f"Tensor {tensor_name}: Best Rank = {optimal_rank}. Last iter results: Frobenius Error = {frobenius_error:.4f}%, Compression Ratio = {compression_ratio:.4f}%"
                    )
                except Exception as e:
                    print(f"Error with method {method}: {e}")
            else:
                print(f"Skipping tensor: {tensor_name} cause rank is exist")
        except Exception as e:
            print(f"Error processing tensor {tensor_name}: {e}")
        finally:
            torch.cuda.empty_cache()
            gc.collect()

Skipping tensor: image-0 cause rank is exist
Skipping tensor: image-1 cause rank is exist
Skipping tensor: image-2 cause rank is exist
Skipping tensor: video-0 cause rank is exist
Skipping tensor: video-1 cause rank is exist
Skipping tensor: video-2 cause rank is exist
Skipping tensor: eeg-0 cause rank is exist
Skipping tensor: eeg-1 cause rank is exist


In [87]:
for tensor_name, tensor in input_tensor_train_tensors.items():
    print(tensor_name, tensor.shape)

image-0 (564, 564, 3)
image-1 (412, 620, 3)
image-2 (689, 1195, 3)
video-0 (220, 256, 144, 3)
video-1 (100, 144, 192, 3)
video-2 (237, 144, 256, 3)
eeg-0 (4, 12, 2, 15, 64, 1281)
eeg-1 (3, 1050, 2, 132, 201)


In [88]:
for tensor_name, rank in tensor_tensor_train_rank_mapping.items():
    print(tensor_name, rank)

image-0 [1, 212, 3, 1]
image-1 [1, 233, 2, 1]
image-2 [1, 289, 3, 1]
video-0 [1, 213, 222, 3, 1]
video-1 [1, 83, 331, 3, 1]
video-2 [1, 237, 374, 3, 1]
eeg-0 [1, 4, 48, 96, 720, 1280, 1]
eeg-1 [1, 3, 3150, 3152, 129, 1]


# Theoretical comparison of some libraries and implementations

Some packages which can decompose some dense type of tensors, from [this](https://arxiv.org/pdf/2103.13756) paper


Decomposition methods which used:
1. Canonical Polyadic Decomposition as PARAllel FACtors analysis (aka PARAFAC aka CPD aka CP)
2. Tucker Decomposition
3. Tensor Train
4. some variants of its (Other)

Tensor types:
1. Dense (D)
2. Sparse (S)
3. BlockSparse (BS)
4. Symmetric
5. Supersymmetric

Target system:
1. CPU (C)
2. GPU(G)
3. Distributed Memory (D)



| Method name                                                                             | Decomposition methods implemented | Tensor Type | Platform | Language         | Git | PyPI | Want to check | Checked     |
|-----------------------------------------------------------------------------------------|-----------------------------------|------------|----------|------------------|------|------|--------------|-------------|
| [AdaTM](https://github.com/hpcgarage/AdaTM)                                             | CP                                | S          | C        | C                | +    | ?    |              |             |
| [BTAS](https://github.com/ValeevGroup/BTAS)                                             | CP, Tucker                        | nan        | C        | C++              | +    | ?    |              |             |
| [CP-CALS](https://github.com/HPAC/CP-CALS)                                              | CP, Other                         | D          | C, G     | C++, Mat         | +    |      | +            |             |
| [CSTF](https://github.com/ZacBlanco/cstf)                                               | Other                             | S          | D        | Scala            | +    | ?    |              |             |
| [D-Tucker](https://datalab.snu.ac.kr/dtucker/resources/DTucker-v1.0.tar.gz)             | Tucker, Other                     | D          | C        | Matlab           |      | ?    |              |             |
| [DFacTo](http://www.joonheechoi.com/research.)                                          | CP                                | S          | C, D     | C++              |      | ?    |              |             |
| [EXATN](https://github.com/ORNL-QCI/exatn)                                              | TensorTrain                       | D          | C, D, G  | C++, Py          | +    |      | +            |             |
| [Genten](https://gitlab.com/tensors/genten)                                             | CP                                | D, S       | C, G     | C++              | +    |      | +            |             |
| GigaTensor                                                                              | CP                                | D          | C        | C++, Python      |      | ?    |              |             |
| [ITensor](https://github.com/ITensor/ITensor)                                           | TensorTrain                       | D, BS      | C, G     | C++, Julia       | +    |      | +            |             |
| [multiway](https://cran.r-project.org/web/packages/multiway/index.html)                 | CP, Tucker, Other                 | D          | C        | R                |      | ?    |              |             |
| [N-way toolbox](http://www.models.life.ku.dk/nwaytoolbox/download)                      | CP, Tucker, Other                 | D          | C        | Matlab           |      | ?    |              |             |
| [ParCube](https://www.cs.ucr.edu/~epapalex/src/parCube.zip)                             | CP                                | S          | C        | Matlab           |      | ?    |              |             |
| [ParTensor](https://github.com/neurocom/partensor-toolbox)                              | CP                                | D          | C, G     | C++              | +    |      | +            |             |
| [ParTI!](https://github.com/hpcgarage/ParTI)                                            | CP, Tucker                        | S          | C, G     | C, CUDA, Mat     | +    | ?    |              |             |
| [PLANC](https://github.com/ramkikannan/planc)                                           | CP                                | S          | C, D     | C++              | +    | ?    |              |             |
| [PLS toolbox](https://eigenvector.com/software/pls-toolbox/)                            | CP          , Tucker              | D          | C        | Matlab           |      | ?    |              |             |
| [Pytensor](https://code.google.com/archive/p/pytensor/source/default/source)            | Tucker                            | D, S       | C        | Python           |      | ?    |              |             |
| [rTensor](https://github.com/jamesyili/rTensor)                                         | CP, Tucker, Other                 | D          | C        | R                | +    |      | +            |             |
| [rTensor (randomized)](https://github.com/erichson/rTensor)                             | CP                                | D          | C        | Python           | +    |      | +       +    |             |
| [scikit-tensor](https://github.com/mnick/scikit-tensor)                                 | CP, Tucker, Other                 | D, S       | C        | Python           | +    | +    | +    +   +   | too old     |
| [Scikit-TT](https://github.com/PGelss/scikit_tt)                                        | TensorTrain                       | D          | C        | Python           | +    |      |     +   +    |             |
| [SPALS](https://github.com/dehuacheng/SpAls)                                            | CP                                | S          | C        | C++              | +    | ?    |              |             |
| [SPARTan](https://github.com/kperros/SPARTan)                                           | Other                             | S          | C        | Matlab           | +    | ?    |              |             |
| [SPLATT](https://github.com/ShadenSmith/splatt)                                         | CP                                | S          | C, D     | C, C++, Oct, Mat | +    | ?    |              |             |
| [SuSMoST](https://susmost.com/downloads.html)                                           | TensorTrain, Other                | D          | C        | Python           |      | ?    |              |             |
| [T3F](https://github.com/Bihaqo/t3f)                                                    | TensorTrain                       | D          | C, G     | Python           | +    | +    | +    + +     | in progress |
| [TDALAB](https://github.com/andrewssobral/TDALAB)                                       | CP                                | D, S       | C        | Python, Matlab   | +    |      | +         +  |             |
| [TeNPy](https://github.com/tenpy/tenpy)                                        | TensorTrain                       | D          | C        | Python           | +    | +    | +      + +   | in progress |
| [Tensor Fox](https://github.com/felipebottega/Tensor-Fox)                               | CP                                | D, S       | C        | Python, Matlab   | +    | +    | +    + +     | ?           |
| [Tensor package](http://www.gipsa-lab.fr/~pierre.comon/TensorPackage/tensorPackage.html) | CP                                | D          | C        | Matlab           |      | ?    |              |             |
| [Tensor Toolbox](https://gitlab.com/tensors/tensor_toolbox)                             | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [tensor_decomposition](https://github.com/cyclops-community/tensor_decomposition)       | CP, Tucker                        | D          | C, D     | Python           | +    |      | +        +   |             |
| [TensorBox](https://github.com/phananhhuy/TensorBox)                                    | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [TensorD](https://github.com/Large-Scale-Tensor-Decomposition/tensorD)                  | CP, Tucker                        | D          | C, G     | Python           | ?    | ?    |              |             |
| [TensorLab](https://www.tensorlab.net)                                                  | CP, Tucker, Other                 | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLab+](https://www.tensorlabplus.net)                                             | CP, Other                         | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLy](https://github.com/tensorly/tensorly)                                        | CP, Tucker, TensorTrain, Other    | D          | C, G     | Python           | +    | +    | +       + +  | in progress |
| [Three-Way](https://github.com/cran/ThreeWay)                                           | CP, Tucker                        | D          | C        | R                | +    |      | +            |             |
| [TNR](https://github.com/ycyuustc/matlab)                                               | Other                             | D          | C        | Matlab           | +    |      | +            |             |
| [TT-Toolbox](https://github.com/oseledets/TT-Toolbox)                                   | TensorTrain                       | D          | C, D, G  | Matlab, Python   | +    |      | +       +    |             |
| [xerus](https://git.hemio.de/xerus/xerus/)                                              | TensorTrain                       | D, S       | C        | C++              | +    |      | +            |             |

# Implementations of Decompositions methods

## TensorLy

In [89]:
# {‘numpy’, ‘mxnet’, ‘pytorch’, ‘tensorflow’, ‘cupy’}
# backend variants for tensorly
# tl.set_backend('pytorch')
# with tl.backend_context(‘pytorch’): ... pass

# video_frames_cuda = tl.tensor(video_frames.copy()).to(device)
# video_frames_cuda = tl.tensor(video_frames.copy())

### Tucker (tl.decomposition.tucker)

In [90]:
logs = LogReader.load_logs_from_file(log_file_path)

Файл ../.cache/method_logs.json не найден.


#### Params

In [91]:
tl.SVD_FUNS

['truncated_svd', 'symeig_svd', 'randomized_svd']

In [92]:
input_tensors = input_tucker_tensors

tensor_rank_mapping = tensor_tucker_rank_mapping

n_iter_max_param = 100

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

init_params = ["svd", "random"]

backend_params = ["pytorch"]

random_state_param = 42

total_iterations = len(list(product(svd_params, init_params, backend_params))) * len(input_tensors)

#### Compare method with some params and log it

In [93]:
for data_type_name_with_index, input_tensor in input_tensors.items():
    rank_param = tensor_rank_mapping[data_type_name_with_index]
    data_type_name = data_type_name_with_index.split("-")[0]

    for backend, svd_func, init_method in tqdm(
            product(backend_params, svd_params, init_params),
            desc="Проверка набора параметров",
            total=total_iterations,
    ):
        library_method_name = "TensorLy_Tucker"
        method_name = f"{library_method_name}_{data_type_name_with_index}_{backend}_{svd_func}_{init_method}"

        print(f"Current method: {method_name}")

        if logs:
            existing_log = next(
                (
                    log
                    for log in logs
                    if log["method_name"] == method_name
                       and log["method_args"].get("init") == init_method
                       and log["method_args"].get("svd") == svd_func
                       and log["qualitative_metrics"].get("TensorLy backend") == backend
                ),
                None,
            )
            if existing_log:
                print(f"Пропущена итерация: логи уже существуют для {method_name}")
                continue

        try:
            with tl.backend_context(backend):
                if backend == "pytorch":
                    tensor_param = tl.tensor(input_tensor).to(torch_device)
                elif backend == "numpy" or backend is None:
                    tensor_param = tl.tensor(input_tensor)

                method_runner = MethodRunner(
                    func=tl.decomposition.tucker,
                    method_input_tensor=tensor_param,
                    library_method_name=library_method_name,
                    backend_name=backend,
                    gpu_memory_tracker=GPUTorchMemoryTracker(),
                    ram_memory_tracker=RAMMemoryTracker(),
                    time_tracker=TimeTracker(),
                )

                method_logger = MethodLogger(
                    method_name=method_name,
                    qualitative_metrics={
                        "Language": "Python",
                        "Library": "TensorLy",
                        "TensorLy backend": f"{backend}",
                        "Tensor type": "Dense",
                        "Data type": data_type_name_with_index,
                        "Platform": "CPU, GPU",
                        "Decomposition method": "Tucker",
                    },
                    method_args={
                        "tensor": tensor_param,
                        "rank": rank_param,
                        "n_iter_max": n_iter_max_param,
                        "init": init_method,
                        "svd": svd_func,
                        "random_state": random_state_param,
                    },
                    runner=method_runner,
                    is_test=False,
                )

                method_logger.run_experiments()

            reconstructed_tensor = method_runner.reconstructed_tensor

            reconstructed_frames = []

            reconstructed_tensor = reconstructed_tensor.cpu().numpy() if backend == "pytorch" else reconstructed_tensor
            reconstructed_frames = np.array([normalize_frames(frame) for frame in reconstructed_tensor])

            save_params_combinations = {
                "image": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                },
                "video": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                    "fps": videos[data_type_name_with_index]["original_fps"] if data_type_name == "video" else 1,
                    "frame_size": videos[data_type_name_with_index]["frame_size"]
                    if data_type_name == "video"
                    else (input_tensor.shape[0], input_tensor.shape[1]),
                },
                "eeg": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                },
            }
            save_params = save_params_combinations[data_type_name]

            SaveFramesFactory.get_save_methods(frame_name=data_type_name).save_frames(**save_params)

        except (torch.cuda.OutOfMemoryError, MemoryError) as e:
            error_message = f"Пропущена итерация из-за ошибки: {backend}, {svd_func}, {init_method}. Ошибка: {e!s}"
            print(error_message)
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            gc.collect()

            method_logger.error_message = error_message

            continue

        finally:
            method_logger.save_logs_to_file(is_test=False)

Проверка набора параметров:   0%|          | 0/48 [00:00<?, ?it/s]

Current method: TensorLy_Tucker_image-0_pytorch_truncated_svd_svd



Эксперимент набора параметров:   0%|          | 0/5 [00:00<?, ?it/s]/home/johndoe_19/git-projects/tensor-methods-comparison/.venv/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:293: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(

Проверка набора параметров:   2%|▏         | 1/48 [00:24<18:55, 24.15s/it]

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_image-0_pytorch_truncated_svd_svd.jpg
Current method: TensorLy_Tucker_image-0_pytorch_truncated_svd_random



Проверка набора параметров:   4%|▍         | 2/48 [00:59<23:41, 30.90s/it]

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_image-0_pytorch_truncated_svd_random.jpg
Current method: TensorLy_Tucker_image-0_pytorch_symeig_svd_svd



Проверка набора параметров:   4%|▍         | 2/48 [01:04<24:48, 32.36s/it]


KeyboardInterrupt: 

#### Clear cache and gc collect

In [31]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

variables_to_delete = [
    "method_runner",
    "method_logger",
    "tensor_param",
    "logs",
    "tensor_rank_mapping",
    "input_tensors",
    "n_iter_max_param",
    "svd_params",
    "init_params",
    "backend_params",
    "random_state_param",
    "total_iterations",
    "data_type_name",
    "input_tensor",
    "rank_param",
    "backend",
    "svd_func",
    "init_method",
    "library_method_name",
    "method_name",
    "reconstructed_tensor",
    "reconstructed_frames",
    "save_params_combinations",
    "save_params",
    "frame",
]

for var in variables_to_delete:
    with contextlib.suppress(KeyError):
        del globals()[var]

gc.collect()

1932

### Tensor Train - MPS (tensorly.decomposition.tensor_train)

In [32]:
logs = LogReader.load_logs_from_file(log_file_path)


KeyboardInterrupt



#### Params

In [33]:
# input_tensors = {
#     "eeg-0": eegs["eeg-0"]["frames"].copy().astype(np.float32),
#     # "eeg-1": eegs["eeg-1"]["frames"].copy().astype(np.float32),
#     "image-2": images["image-2"]["frames"].copy().astype(np.float32),
#     "image-1": images["image-1"]["frames"].copy().astype(np.float32),
#     "image-0": images["image-0"]["frames"].copy().astype(np.float32),
#     "video-2": videos["video-2"]["frames"].copy().astype(np.float32),
#     "video-1": videos["video-1"]["frames"].copy().astype(np.float32),
#     "video-0": videos["video-0"]["frames"].copy().astype(np.float32),
# }
# 
# tensor_rank_mapping = {
#     "image-0": [1, 212, 212, 1],
#     "image-1": [1, 170, 212, 1],
#     "image-2": [1, 290, 300, 1],
#     "video-0": [1, 300, 215, 300, 1],
#     "video-1": [1, 500, 278, 500, 1],
#     "video-2": [1, 500, 375, 500, 1],
#     "eeg-0": [1, 1000, 1750, 2000, 1750, 1000, 1],  # (4, 12, 3, 29, 64, 321)
#     # "eeg-1": [1, 1000, 1750, 2000, 1750, 1000, 1],  # (4, 12, 3, 29, 64, 321)
# }

input_tensors = input_tensor_train_tensors

tensor_rank_mapping = tensor_tensor_train_rank_mapping

svd_params = ["randomized_svd", "truncated_svd", "symeig_svd"]

backend_params = ["pytorch"] # , "numpy"

total_iterations = len(list(product(backend_params, svd_params))) * len(input_tensors)

#### Compare method with some params and log it

In [ ]:
for data_type_name_with_index, input_tensor in input_tensors.items():
    rank_param = tensor_rank_mapping[data_type_name_with_index]
    data_type_name = data_type_name_with_index.split("-")[0]

    for backend, svd_func in tqdm(
            product(backend_params, svd_params),
            desc="Проверка набора параметров",
            total=total_iterations,
    ):
        library_method_name = "TensorLy_TensorTrain"
        method_name = f"{library_method_name}_{data_type_name_with_index}_{backend}_{svd_func}"

        print(f"Current method: {method_name}")

        if logs:
            existing_log = next(
                (
                    log
                    for log in logs
                    if log["method_name"] == method_name
                       and log["method_args"].get("svd") == svd_func
                       and log["qualitative_metrics"].get("TensorLy backend") == backend
                ),
                None,
            )
            if existing_log:
                print(f"Пропущена итерация: логи уже существуют для {method_name}")
                continue

        try:
            with tl.backend_context(backend):
                if backend == "pytorch":
                    tensor_param = tl.tensor(input_tensor).to(torch_device)
                elif backend == "numpy" or backend is None:
                    tensor_param = tl.tensor(input_tensor)

                method_runner = MethodRunner(
                    func=tl.decomposition.tensor_train,
                    method_input_tensor=tensor_param,
                    library_method_name=library_method_name,
                    backend_name=backend,
                    gpu_memory_tracker=GPUTorchMemoryTracker(),
                    ram_memory_tracker=RAMMemoryTracker(),
                    time_tracker=TimeTracker(),
                )

                method_logger = MethodLogger(
                    method_name=method_name,
                    qualitative_metrics={
                        "Language": "Python",
                        "Library": "TensorLy",
                        "TensorLy backend": f"{backend}",
                        "Tensor type": "Dense",
                        "Data type": data_type_name_with_index,
                        "Platform": "CPU, GPU",
                        "Decomposition method": "TensorTrain",
                    },
                    method_args={
                        "input_tensor": tensor_param,
                        "rank": rank_param,
                        "svd": svd_func,
                    },
                    runner=method_runner,
                    is_test=False,
                )

                method_logger.run_experiments()

            reconstructed_tensor_from_tt_factors = method_runner.reconstructed_tensor

            reconstructed_frames = []

            for tt_factor in reconstructed_tensor_from_tt_factors:
                if backend == "pytorch":
                    reconstructed_frames.append(normalize_frames(tt_factor.cpu().numpy()))
                else:
                    reconstructed_frames.append(normalize_frames(tt_factor))
            reconstructed_frames = np.array(reconstructed_frames)

            save_params_combinations = {
                "image": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                },
                "video": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                    "fps": videos[data_type_name_with_index]["original_fps"] if data_type_name == "video" else 1,
                    "frame_size": videos[data_type_name_with_index]["frame_size"]
                    if data_type_name == "video"
                    else (input_tensor.shape[0], input_tensor.shape[1]),
                },
                "eeg": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                },
            }
            save_params = save_params_combinations[data_type_name]

            SaveFramesFactory.get_save_methods(frame_name=data_type_name).save_frames(**save_params)

            # break

        except (torch.cuda.OutOfMemoryError, MemoryError) as e:
            error_message = f"Пропущена итерация из-за ошибки: {backend}, {svd_func}. Ошибка: {e!s}"
            print(error_message)
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            gc.collect()

            method_logger.error_message = error_message

            continue
        finally:
            method_logger.save_logs_to_file(is_test=False)
    # break

#### Clear cache and gc collect

In [35]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

variables_to_delete = [
    "logs",
    "tensor_rank_mapping",
    "input_tensors",
    "svd_params",
    "backend_params",
    "total_iterations",
    "data_type_name",
    "input_tensor",
    "rank_param",
    "backend",
    "svd_func",
    "library_method_name",
    "method_name",
    "reconstructed_tensor_from_tt_factors",
    "method_runner",
    "method_logger",
    "reconstructed_frames",
    "save_params_combinations",
    "save_params",
    "tt_factor",
]

for var in variables_to_delete:
    with contextlib.suppress(KeyError):
        del globals()[var]

gc.collect()

485

### CANDECOMP/PARAFAC (tensorly.decomposition.parafac)

In [26]:
logs = LogReader.load_logs_from_file(log_file_path)

#### params

In [27]:
input_tensors = {
    "image-2": images["image-2"]["frames"].copy().astype(np.float32),
    "image-0": images["image-0"]["frames"].copy().astype(np.float32),
    "image-1": images["image-1"]["frames"].copy().astype(np.float32),
    # "video-0": videos['video-0']['frames'].copy().astype(np.float32),
    # "video-1": videos['video-1']['frames'].copy().astype(np.float32),
    # "video-2": videos['video-2']['frames'].copy().astype(np.float32),
}

tensor_rank_mapping = {
    "image-0": 422,
    "image-1": 370,
    "image-2": 655,
    # "video-0": 3500,
    # "video-1": 2036,
    # "video-2": 3620,
}

# backend_params = ["pytorch", "numpy"]
backend_params = ["pytorch"]

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

init_params = ["random", "svd"]

normalize_factors_params = [False, True]
# normalize_factors_params = [False]

orthogonalise_params = [False, True]
# orthogonalise_params = [False]

linesearch_params = [False, True]

cvg_criterion_params = ["abs_rec_error", "rec_error"]

l2_reg_params = [0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0]
# l2_reg_params = [0]

tol_params = [1e-8, 1e-5, 1e-6, 1e-7, 1e-9]
# tol_params = [1e-8]

n_iter_max_param = 100

random_state_param = 42

total_iterations = len(
    list(
        product(
            backend_params,
            svd_params,
            init_params,
            normalize_factors_params,
            orthogonalise_params,
            linesearch_params,
            cvg_criterion_params,
            l2_reg_params,
            tol_params,
        )
    )
) * len(input_tensors)

#### Compare method with some params and log it

In [ ]:
for data_type_name_with_index, input_tensor in input_tensors.items():
    rank_param = tensor_rank_mapping[data_type_name_with_index]
    data_type_name = data_type_name_with_index.split("-")[0]

    for (
            backend,
            normalize_factors,
            orthogonalise,
            linesearch,
            cvg_criterion,
            l2_reg,
            tol,
            init_method,
            svd_func,
    ) in tqdm(
        product(
            backend_params,
            normalize_factors_params,
            orthogonalise_params,
            linesearch_params,
            cvg_criterion_params,
            l2_reg_params,
            tol_params,
            init_params,
            svd_params,
        ),
        desc="Проверка набора параметров",
        total=total_iterations,
    ):
        library_method_name = "TensorLy_CP"
        method_name = f"{library_method_name}_{data_type_name_with_index}_{backend}_{svd_func}_{init_method}_{normalize_factors}_{orthogonalise}_{cvg_criterion}_{l2_reg}_{tol}"

        print(f"Current method: {method_name}")

        if logs:
            existing_log = next(
                (
                    log
                    for log in logs
                    if log["method_name"] == method_name
                       and log["method_args"].get("init") == init_method
                       and log["method_args"].get("svd") == svd_func
                       and log["method_args"].get("normalize_factors") == normalize_factors
                       and log["method_args"].get("orthogonalise") == orthogonalise
                       and log["method_args"].get("cvg_criterion") == cvg_criterion
                       and log["method_args"].get("l2_reg") == l2_reg
                       and log["method_args"].get("tol") == tol
                       and log["qualitative_metrics"].get("TensorLy backend") == backend
                       and log["method_args"].get("linesearch") == linesearch
                ),
                None,
            )
            if existing_log:
                print(f"Пропущена итерация: логи уже существуют для {method_name}")
                continue

        try:
            with tl.backend_context(backend):
                if backend == "pytorch":
                    tensor_param = tl.tensor(input_tensor).to(torch_device)
                elif backend == "numpy" or backend is None:
                    tensor_param = tl.tensor(input_tensor)

                method_runner = MethodRunner(
                    func=tl.decomposition.parafac,
                    method_input_tensor=tensor_param,
                    library_method_name=library_method_name,
                    backend_name=backend,
                    gpu_memory_tracker=GPUTorchMemoryTracker(),
                    ram_memory_tracker=RAMMemoryTracker(),
                    time_tracker=TimeTracker(),
                )

                method_logger = MethodLogger(
                    method_name=method_name,
                    qualitative_metrics={
                        "Language": "Python",
                        "Library": "TensorLy",
                        "TensorLy backend": f"{backend}",
                        "Tensor type": "Dense",
                        "Data type": data_type_name_with_index,
                        "Platform": "CPU, GPU",
                        "Decomposition method": "CP",
                    },
                    method_args={
                        "tensor": tensor_param,
                        "rank": rank_param,
                        "n_iter_max": n_iter_max_param,
                        "init": init_method,
                        "svd": svd_func,
                        "normalize_factors": normalize_factors,
                        "orthogonalise": orthogonalise,
                        "tol": tol,
                        "random_state": random_state_param,
                        "l2_reg": l2_reg,
                        "cvg_criterion": cvg_criterion,
                        "linesearch": linesearch,
                    },
                    runner=method_runner,
                    is_test=False,
                )

                method_logger.run_experiments()

            reconstructed_tensor = method_runner.reconstructed_tensor

            reconstructed_frames = []

            reconstructed_tensor = reconstructed_tensor.cpu().numpy() if backend == "pytorch" else reconstructed_tensor
            reconstructed_frames = np.array([normalize_frames(frame) for frame in reconstructed_tensor])

            save_params_combinations = {
                "image": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                },
                "video": {
                    "name": method_logger.name,
                    "frames": reconstructed_frames,
                    "fps": videos[data_type_name_with_index]["original_fps"] if data_type_name == "video" else 1,
                    "frame_size": videos[data_type_name_with_index]["frame_size"]
                    if data_type_name == "video"
                    else (input_tensor.shape[0], input_tensor.shape[1]),
                },
            }
            save_params = save_params_combinations[data_type_name]

            SaveFramesFactory.get_save_methods(frame_name=data_type_name).save_frames(**save_params)

        except (
                torch.cuda.OutOfMemoryError,
                MemoryError,
                np.linalg.LinAlgError,
                torch._C._LinAlgError,
        ) as e:
            error_message = f"Пропущена итерация из-за ошибки: {backend}, {svd_func}, {init_method}, {normalize_factors}, {orthogonalise}, {tol}, {l2_reg}, {cvg_criterion}. Ошибка: {e!s}"
            print(error_message)
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            gc.collect()

            method_logger.error_message = error_message

            continue
        finally:
            method_metrics = method_logger.save_logs_to_file(is_test=False)

#### Clear cache and gc collect

In [29]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

variables_to_delete = [
    "method_runner",
    "method_logger",
    "tensor_param",
    "logs",
    "tensor_rank_mapping",
    "input_tensors",
    "n_iter_max_param",
    "svd_params",
    "init_params",
    "backend_params",
    "random_state_param",
    "total_iterations",
    "data_type_name",
    "input_tensor",
    "rank_param",
    "backend",
    "svd_func",
    "init_method",
    "library_method_name",
    "method_name",
    "reconstructed_tensor",
    "reconstructed_frames",
    "save_params_combinations",
    "save_params",
    "frame",
    "normalize_factors_params",
    "orthogonalise_params",
    "cvg_criterion_params",
    "l2_reg_params",
    "tol_params",
    "normalize_factors",
    "orthogonalise",
    "cvg_criterion",
    "tol",
]

for var in variables_to_delete:
    with contextlib.suppress(KeyError):
        del globals()[var]

gc.collect()

485

## T3F

### Tensor Train

In [61]:
logs = LogReader.load_logs_from_file(log_file_path)

#### Params

In [62]:
# input_tensors = {
#     "image-2": images["image-2"]["frames"].copy().astype(np.float32),
#     "image-1": images["image-1"]["frames"].copy().astype(np.float32),
#     "image-0": images["image-0"]["frames"].copy().astype(np.float32),
#     "video-2": videos["video-2"]["frames"].copy().astype(np.float32),
#     "video-1": videos["video-1"]["frames"].copy().astype(np.float32),
#     "video-0": videos["video-0"]["frames"].copy().astype(np.float32),
# }
# 
# tensor_rank_mapping = {
#     "image-0": [1, 212, 212, 1],
#     "image-1": [1, 170, 212, 1],
#     "image-2": [1, 290, 300, 1],
#     "video-0": [1, 300, 215, 300, 1],
#     "video-1": [1, 500, 278, 500, 1],
#     "video-2": [1, 500, 375, 500, 1],
# }

input_tensors = input_tensor_train_tensors

tensor_rank_mapping = tensor_tensor_train_rank_mapping

backend_param = "tensorflow"

total_iterations = len(input_tensors)

#### Compare method with some params and log it

In [63]:
for data_type_name_with_index, input_tensor in tqdm(
        input_tensors.items(),
        desc="Проверка набора параметров",
        total=total_iterations,
):
    rank_param = tensor_rank_mapping[data_type_name_with_index]
    data_type_name = data_type_name_with_index.split("-")[0]
    tensor_param = input_tensor

    library_method_name = "T3F_TensorTrain"
    method_name = f"{library_method_name}_{data_type_name_with_index}"

    print(method_name)

    if logs:
        existing_log = next((log for log in logs if log["method_name"] == method_name), None)
        if existing_log:
            print(f"Пропущена итерация: логи уже существуют для {method_name}")
            continue

    gc.collect()

    method_runner = MethodRunner(
        func=t3f.to_tt_tensor,
        method_input_tensor=tensor_param,
        library_method_name=library_method_name,
        backend_name=backend_param,
        gpu_memory_tracker=GPUTensorflowMemoryTracker(tf_devices=tf_devices),
        ram_memory_tracker=RAMMemoryTracker(),
        time_tracker=TimeTracker(),
    )

    method_logger = MethodLogger(
        method_name=method_name,
        qualitative_metrics={
            "Language": "Python",
            "Library": "T3F",
            "T3F backend": f"{backend_param}",
            "Tensor type": "Dense",
            "Data type": data_type_name_with_index,
            "Platform": "CPU, GPU",
            "Decomposition method": "TensorTrain",
        },
        method_args={
            "tens": tensor_param,
            "max_tt_rank": rank_param,
        },
        runner=method_runner,
        is_test=False,
    )

    method_logger.run_experiments()

    reconstructed_tensor_t3f = method_runner.reconstructed_tensor

    method_logger.save_logs_to_file(is_test=False)

    reconstructed_frames = []

    for frame in reconstructed_tensor_t3f:
        reconstructed_frames.append(normalize_frames(frame))
    reconstructed_frames = np.array(reconstructed_frames)

    save_params_combinations = {
        "image": {"name": method_logger.name, "frames": reconstructed_frames},
        "video": {
            "name": method_logger.name,
            "frames": reconstructed_frames,
            "fps": videos[data_type_name_with_index]["original_fps"] if data_type_name == "video" else 1,
            "frame_size": videos[data_type_name_with_index]["frame_size"]
            if data_type_name == "video"
            else (input_tensor.shape[0], input_tensor.shape[1]),
        },
    }

    save_params = save_params_combinations[data_type_name]

    SaveFramesFactory.get_save_methods(frame_name=data_type_name).save_frames(**save_params)

Проверка набора параметров:   0%|          | 0/8 [00:00<?, ?it/s]

T3F_TensorTrain_image-0



Эксперимент набора параметров:   0%|          | 0/5 [00:00<?, ?it/s]I0000 00:00:1750014896.982830  265932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750014896.983058  265932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750014896.983142  265932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750014897.007972  265932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1750014897.009182  265932 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/T3F_TensorTrain_image-0.jpg
T3F_TensorTrain_image-1



Проверка набора параметров:  25%|██▌       | 2/8 [00:33<01:36, 16.16s/it]

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/T3F_TensorTrain_image-1.jpg
T3F_TensorTrain_image-2



Эксперимент набора параметров:   0%|          | 0/5 [00:00<?, ?it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f0601b09a10>>
Traceback (most recent call last):
  File "/home/johndoe_19/git-projects/tensor-methods-comparison/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Process MemTimer-30:
Traceback (most recent call last):
  File "/root/.local/share/uv/python/cpython-3.11.10-linux-x86_64-gnu/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Эксперимент набора параметров:   0%|          | 0/5 [00:01<?, ?it/s]  File "/home/johndoe_19/git-projects/tensor-methods-comparison/.venv/lib/python3.11/site-packages/memory_profiler.py", line 262, in run
    stop = self.pipe.poll(self.interval)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Проверка набора параметров:  25%|██

KeyboardInterrupt: 

#### Clear cache and gc collect

In [25]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

numba_device = cuda.get_current_device()
numba_device.reset()

variables_to_delete = [
    "logs",
    "tensor_rank_mapping",
    "input_tensors",
    "backend_param",
    "total_iterations",
    "data_type_name",
    "input_tensor",
    "rank_param",
    "tensor_param",
    "library_method_name",
    "method_name",
    "method_runner",
    "method_logger",
    "reconstructed_tensor_t3f",
    "reconstructed_frames",
    "frame",
    "save_params_combinations",
    "save_params",
]

for var in variables_to_delete:
    with contextlib.suppress(KeyError):
        del globals()[var]

gc.collect()

19

## TeNPy (not implemented)

### Params

In [26]:
# tensor_param = video_frames.copy().astype(np.float32)
#
# # Размерность физического индекса
# d = tensor_param.shape[-1]  # в вашем случае это 3
#
# # Создаем объект LegCharge
# leg = LegCharge.from_trivial(d)
#
# # Создаем объекты Site для каждого физического индекса, кроме последнего
# sites = [Site(leg) for _ in range(tensor_param.ndim - 1)]
#
# rank_param = [1, 500, 302, 500, 1]

### Implementation

In [27]:
# mps = MPS.from_full(sites=sites, psi=tensor_param, normalize=False)
#
# reconstructed_tensor = mps.to_full_tensor()

In [28]:
# logs = load_logs_from_file(log_file_path)
#
# method_name = f"TeNPy_TensorTrain"
#
# if logs:
#     existing_log = next(
#         (log for log in logs if log['method_name'] == method_name),
#         None
#     )
#     if existing_log:
#         error_message = f"Пропущена итерация: логи уже существуют для {method_name}"
#         raise error_message
#
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.synchronize()
#
# method_logs = MethodLogger(
#     method_name=method_name,
#     method_input_tensor=tensor_param,
#     qualitative_metrics={
#         "Language": "Python",
#         "Library": "TeNPy",
#         "Tensor type": "Dense",
#         "Platform": "CPU",
#         "Decomposition method": "TensorTrain",
#     },
#     method_args={
#         "sites": tensor_param,
#         "psi": rank_param,
#         "normalize": False,
#     },
#     func=tenpy.networks.mps.MPS.from_full
# )
#
# method_logs_list.append(method_logs)
#
# tt_factors = method_logs.method_result
#
# reconstruct_frames_from_tenpy_tt_factors = tt_factors.to_full_tensor()

In [29]:
# method_logs.quantitative_metrics['compression_ratio'] = (100.0 * get_tensors_size(*tt_factors) / get_tensors_size(tensor_param))

# method_logs.quantitative_metrics['frobenius_error'] = (
#         100.0 * np.linalg.norm(reconstruct_frames_from_tenpy_tt_factors - tensor_param) / tl.norm(
#         tensor_param))
#
# save_logs_to_file(method_logs=method_logs, is_test=True)
#
# reconstructed_frames = []
#
# for i in range(len(reconstruct_frames_from_tenpy_tt_factors)):
#     reconstructed_frames.append(normalize_frame_tensorly_tensortrain(reconstruct_frames_from_tenpy_tt_factors[i]))
#
# save_frames_as_video(name=method_logs.name, frames=reconstructed_frames, fps=original_fps, frame_size=frame_size)